## Observation Maker

1.  Enter source data in csv file to simulate patient generated wt observations

 - Hard Code: 

Patient use existing Synthea patient: Eldon718 Halvorson124  Patient/06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b

 - Variables: 

    - measurement
    - datetime
    - context ? ( see OMH )
    - device info
    - sensed or entered
    - SR Key?
    
- Add Reliability Tags 
- Add example name and description
- Validate
- Save

In [23]:
%cd '/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker'

/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker


###  Import from YAML file Observation template

In [24]:
from fhir.resources.observation import Observation
from fhir.resources import construct_fhir_element

In [25]:
obs_obj = Observation.parse_file("Observation_wt.yml")

#for obj in obs_obj:
    #print(obj)
my_type = next(i.type_.__resource_type__ for i in obs_obj.element_properties() if i.name == "basedOn")
my_type

'Reference'

### function to add complex elements to resource

In [26]:
def update_obs(element,element_dict, resource = obs_obj):
    element_type = next(i.type_.__resource_type__ for i in resource.element_properties() if i.name == element)
    print(element,element_dict, element_type)
    new_element=construct_fhir_element(element_type,element_dict)
    try: # assume not a list 
        setattr(resource,element,new_element) # not a list 
        print(getattr(resource,element).yaml())
    except: # treat as list
        try: # assume list already exists
            getattr(resource,element).append(new_element)
        except: # list does not exist
            setattr(resource,element,[new_element])
        print(getattr(resource,element)[0].yaml())

### update the effective dataTime

In [27]:
from datetime import datetime, timezone, timedelta

timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
now = datetime.now(tzinfo)
now

datetime.datetime(2021, 7, 1, 16, 36, 30, 968837, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600)))

In [28]:
obs_obj.effectiveDateTime = now
print(obs_obj.effectiveDateTime)

2021-07-01 16:36:30.968837-08:00


### update the encounter

what code or codes should be used for this  ( "at home measuremet")?

todo ... review patent empowerment work.

for now diplay only

In [29]:
display = "At Home Monitoring"
element = 'encounter'
element_dict = dict(
            #reference = reference,
            #identifier = dict(value = identifier),  # business identifier
            display = display,
            )

update_obs(element,element_dict)

encounter {'display': 'At Home Monitoring'} Reference
display: At Home Monitoring



### update the value

 for now assume in pounds and only update the decimal value

In [30]:
my_wt = 160
obs_obj.valueQuantity.value = my_wt
print(obs_obj.valueQuantity.yaml())

value: 160.0
unit: pounds
system: http://unitsofmeasure.org
code: '[lb_av]'



### Add Device info which provides data provenance

options

- fhir id
- identifier  ( can this be overloaded to indicate the identifier for device without doing a reference ? )
- string 

In [31]:
reference = 'Device/123'
identifier = 'Dev123456'  # business identifier
display = 'WiTscale S200 Bluetooth scale'
element = 'device'
element_dict = dict(
            reference = reference,
            identifier = dict(value = identifier),  # business identifier
            display = display,
            )

update_obs(element,element_dict)

device {'reference': 'Device/123', 'identifier': {'value': 'Dev123456'}, 'display': 'WiTscale S200 Bluetooth scale'} Reference
reference: Device/123
identifier:
  value: Dev123456
display: WiTscale S200 Bluetooth scale



### Add SR Key to `basedOn` element

options 

- fhir id
- identifier ( can this be overloaded to indicate the identifier for device without doing a reference ? )
- string


In [32]:
reference = 'ServiceRequest/123'
identifier = 'SR123456'  # business identifier
display = 'Provider X Home weight monitoring'
element = 'basedOn'
element_dict = dict(
            reference = reference,
            identifier = dict(value = identifier),  # business identifier
            display = display,
            )

update_obs(element,element_dict)

basedOn {'reference': 'ServiceRequest/123', 'identifier': {'value': 'SR123456'}, 'display': 'Provider X Home weight monitoring'} Reference
reference: ServiceRequest/123
identifier:
  value: SR123456
display: Provider X Home weight monitoring



### Add any device/gateway/service created identifier to observation

see OMH for examples

In [33]:
source_identifier = "6f3a11a2-0e34-4ef5-a226-1fd7277e82d0"
element ='identifier'
element_dict = dict(
                value = source_identifier,
                assigner = dict(display=display,)
                )

update_obs(element,element_dict)

identifier {'value': '6f3a11a2-0e34-4ef5-a226-1fd7277e82d0', 'assigner': {'display': 'Provider X Home weight monitoring'}} Identifier
value: 6f3a11a2-0e34-4ef5-a226-1fd7277e82d0
assigner:
  display: Provider X Home weight monitoring



### Add more context to observation

see OMH

In [34]:
#...TODO...

### Add modality extension observation

uri: `http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality`

see [OMH](https://healthedata1.github.io/mFHIR/StructureDefinition-extension-modality.html)

*NOTE: This extension was originally defined to go on the device element, but think is better on the Obervation*

|Code|Display|Definition|
|---|---|---|
|sensed|Sensed|Device measurement is sensed directly by the device|
|self-reported|Self Reported|Device measurement is entered by the user|



In [35]:
url = 'http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality'
modality = "sensed"

element = 'extension'
element_dict = dict(
                valueCode = modality,
                url = url,
                )

update_obs(element,element_dict)

extension {'valueCode': 'sensed', 'url': 'http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality'} Extension
url: http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality
valueCode: sensed



### Add reliability tags to observation

Using Josh's suggested codes as a starting point...

>All data coming via the patient are submitted with a tag like {"system": "http://terminology.hl7.org/CodeSystem/common-tags", "code": "patient-supplied"}
FHIR searches find resources with this tag by default; clients that don't want them can filter them out, through search params
Providers and EHRs can do what they want with these data (e.g., leaving them tagged; reviewing and adding a provider-reviewed tag; reviewing and stripping the patient-supplied tag; deleting them...)

for now need own system:

url = `http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags`

|Code|Display|Definition|
|---|---|---|
|patient-supplied|Patient Supplied Data|Data is supplied by patient - either patient generated data or data generated elsewhere and forwarded by patient (todo get references to definitions of PGD)|
|provider-reviewed|Provider Reviewed Data|Data is supplied by patient and has been reviewed by provider ( either manully or through some automated fashion)|

In [36]:
url = 'http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags'
tag = "patient-supplied"
element = 'tag'
element_dict = dict(system = url,
                            code = tag,
                           )
resource = obs_obj.meta
update_obs(element,element_dict,resource)

tag {'system': 'http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags', 'code': 'patient-supplied'} Coding
system: http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags
code: patient-supplied



### Add description and example name extension to observation

this annotates each example for future documentation

In [37]:
url = 'http://hl7.org/fhir/StructureDefinition/instance-name'
instance_name = "Patient Supplied WT Example 1"
element = 'extension'
element_dict = dict(
                valueString = instance_name,
                url = url,
                )
update_obs(element,element_dict,resource)

extension {'valueString': 'Patient Supplied WT Example 1', 'url': 'http://hl7.org/fhir/StructureDefinition/instance-name'} Extension
url: http://hl7.org/fhir/StructureDefinition/instance-name
valueString: Patient Supplied WT Example 1



In [38]:
url = 'http://hl7.org/fhir/StructureDefinition/instance-description'
instance_description = "This is a simple patient entered weight example for the Argo Write project"
element = 'extension'
element_dict = dict(
                valueMarkdown = instance_description,
                url = url,
                )
update_obs(element,element_dict,resource)

extension {'valueMarkdown': 'This is a simple patient entered weight example for the Argo Write project', 'url': 'http://hl7.org/fhir/StructureDefinition/instance-description'} Extension
url: http://hl7.org/fhir/StructureDefinition/instance-name
valueString: Patient Supplied WT Example 1



In [39]:
print(obs_obj.yaml())

resourceType: Observation
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: Patient Supplied WT Example 1
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a simple patient entered weight example for the Argo Write
      project
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-weight
  tag:
  - system: http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags
    code: patient-supplied
extension:
- url: http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality
  valueCode: sensed
identifier:
- value: 6f3a11a2-0e34-4ef5-a226-1fd7277e82d0
  assigner:
    display: Provider X Home weight monitoring
basedOn:
- reference: ServiceRequest/123
  identifier:
    value: SR123456
  display: Provider X Home weight monitoring
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: vital-signs
    d

### Validate

In [40]:
from requests import post
from IPython.display import display as Display, HTML
def validate(data):

    fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    #fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    r = post(f'{fhir_test_server}/Questionnaire/$validate', headers = headers, data = data)
    return r

print('...validating')
r = validate(obs_obj.json())
r.status_code, r.json()
Display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

...validating


### Save Locally

In [41]:
from pathlib import Path
ig_source_path = 'examples'

path = Path.cwd() / ig_source_path / f'{obs_obj.meta.extension[0].valueString}.yaml'
print(f'...........saving to file {path}............')
path.write_text(obs_obj.yaml())

...........saving to file /Users/ehaas/Documents/Python/MyNotebooks/Obs_maker/examples/Patient Supplied WT Example 1.yaml............


1543